DEFINE VARIABLE, FUNCTIONS FOR CLASSIFICATION

In [1]:
import matplotlib.pyplot as plt
from PIL import Image


import cv2
import numpy as np
import os


In [2]:
import argparse
import time
from pathlib import Path
import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel


from detect import *


In [ ]:
# init OCR
import easyocr

reader = easyocr.Reader(['vi','en'])

In [3]:
# Init YOLO v7

weights     = 'yolov7.pt'
source      = 'inference/images'  # file/folder, 0 for webcam
img_size    = 640
conf_thres  = 0.25  # object confidence threshold
iou_thres   = 0.45  # IOU threshold for NMS
device      = ''  # cuda device, i.e., '0', '0,1,2,3', or 'cpu'
view_img    = False  # display results
save_txt    = False  # save results to *.txt
save_conf   = False  # save confidences in --save-txt labels
nosave      = False  # do not save images/videos
classes     = None  # filter by class, e.g., [0] or [0, 2, 3]
agnostic_nms = False  # class-agnostic NMS
augment     = False  # augmented inference
update      = False  # update all models
project     = 'runs/detect'  # save results to project/name
name        = 'exp'  # save results to project/name
exist_ok    = True  # existing project/name ok, do not increment
no_trace    = False  # don't trace model

In [4]:
source, weights, view_img, save_txt, imgsz, trace = source, weights, view_img, save_txt, img_size, not no_trace
save_img = not nosave and not source.endswith('.txt')  # save inference images
webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
    ('rtsp://', 'rtmp://', 'http://', 'https://'))

# Directories
save_dir = Path(increment_path(Path(project) / name, exist_ok=exist_ok))  # increment run
(save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

# Initialize
set_logging()
device = select_device(device)
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)  # check img_size

if trace:
    model = TracedModel(model, device, img_size)

if half:
    model.half()  # to FP16

# Second-stage classifier
classify = False
if classify:
    modelc = load_classifier(name='resnet101', n=2)  # initialize
    modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()



YOLOR 🚀 v0.1-128-ga207844 torch 2.4.0+cu121 CUDA:0 (NVIDIA Graphics Device, 12008.875MB)

/home/phucnguyen/code/github_clone/YOLO/yolov7/models/experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues re

Fusing layers... 


Model Summary: 306 layers, 36905341 parameters, 36905341 gradients


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
 Convert model to Traced-model... 


/home/phucnguyen/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:811: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  param_grad = param.grad


 traced_script_module saved! 
 model is traced! 



In [5]:
image_path = "/home/phucnguyen/code/github_clone/YOLO/yolov7/inference/images/cccd_resized_standard.jpg"

ALWAYS RESIZE IMAGE TO 1080 X 681 ( AS STANDARD FOR CCCD AND ALSO BEST RESULT IN OCR)

In [6]:
import cv2

def resize_image(input_path, output_path):
    # Đọc ảnh
    image = cv2.imread(input_path)
    
    width = 1080
    height = 651

    # Resize về kích thước mục tiêu
    resized = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)

    # Lưu ảnh đã resize
    cv2.imwrite(output_path, resized)
    print(f"Ảnh đã resize và lưu tại: {output_path}")

# Resize ảnh thành 720p
#resize_image("data/CCCD.jpg", "data/CCCD_resized_720p.jpg", 1280, 720)

# Resize ảnh thành 1080p
#resize_image("data/CCCD.jpg", "data/cccd_resized_1080p.jpg", 1920, 1080)

#resize_image("data/CCCD.jpg", "data/cccd_resized_standard.jpg", 1280, 807)

resize_image(image_path, image_path)  # 1080 x 681


Ảnh đã resize và lưu tại: /home/phucnguyen/code/github_clone/YOLO/yolov7/inference/images/cccd_resized_standard.jpg


CALL YOLO V7 TO DETECT FACE

After this, we will have bounding box of human or not

In [7]:
# Variable to hold
human = False

In [8]:
# Set Dataloader
vid_path, vid_writer = None, None
if webcam:
    view_img = check_imshow()
    cudnn.benchmark = True  # set True to speed up constant image size inference
    dataset = LoadStreams(source, img_size=imgsz, stride=stride)
else:
    dataset = LoadImages(source, img_size=imgsz, stride=stride)

# Get names and colors
names = model.module.names if hasattr(model, 'module') else model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

# Run inference
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
old_img_w = old_img_h = imgsz
old_img_b = 1

/home/phucnguyen/.local/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [11]:
t0 = time.time()
for path, img, im0s, vid_cap in dataset:
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Warmup
    if device.type != 'cpu' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
        old_img_b = img.shape[0]
        old_img_h = img.shape[2]
        old_img_w = img.shape[3]
        for i in range(3):
            model(img, augment=augment)[0]

    # Inference
    t1 = time_synchronized()
    with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
        pred = model(img, augment=augment)[0]
    t2 = time_synchronized()

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes=classes, agnostic=agnostic_nms)
    t3 = time_synchronized()

    # Apply Classifier
    if classify:
        pred = apply_classifier(pred, modelc, img, im0s)

    # Process detections
    for i, det in enumerate(pred):  # detections per image
        if human == True:
            break
        if webcam:  # batch_size >= 1
            p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
        else:
            p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

        p = Path(p)  # to Path
        save_path = str(save_dir / p.name)  # img.jpg
        txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                
                print(s)

            # Write results
            for *xyxy, conf, cls in reversed(det):
                
                print(f"Image size (im0): {im0.shape}")
                if save_txt:  # Write to file
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                    line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                    with open(txt_path + '.txt', 'a') as f:
                        f.write(('%g ' * len(line)).rstrip() % line + '\n')

                if save_img or view_img:  # Add bbox to image
                    label = f'{names[int(cls)]} {conf:.2f}'
                    
                    if names[int(cls)] != "person":
                        continue
                    else:
                        face_bounding_box = [[int(xyxy[0]), int(xyxy[1])], [int(xyxy[2]), int(xyxy[3])]]
                        human = True
                        print("Found Human")
                        break
                        #plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)
                    

        # Print time (inference + NMS)
#         print(f'{s}Done. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS')

#         # Stream results
#         if view_img:
#             cv2.imshow(str(p), im0)
#             cv2.waitKey(1)  # 1 millisecond

#         # Save results (image with detections)
#         if save_img:
#             if dataset.mode == 'image':
#                 cv2.imwrite(save_path, im0)
#                 print(f" The image with the result is saved in: {save_path}")
#             else:  # 'video' or 'stream'
#                 if vid_path != save_path:  # new video
#                     vid_path = save_path
#                     if isinstance(vid_writer, cv2.VideoWriter):
#                         vid_writer.release()  # release previous video writer
#                     if vid_cap:  # video
#                         fps = vid_cap.get(cv2.CAP_PROP_FPS)
#                         w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#                         h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#                     else:  # stream
#                         fps, w, h = 30, im0.shape[1], im0.shape[0]
#                         save_path += '.mp4'
#                     vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
#                 vid_writer.write(im0)

# if save_txt or save_img:
#     s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
#     #print(f"Results saved to {save_dir}{s}")

print(f'Done. ({time.time() - t0:.3f}s)')

1 person, 
1 person, 2 books, 
Image size (im0): (681, 1080, 3)
Image size (im0): (681, 1080, 3)
Image size (im0): (681, 1080, 3)
Found Human
Done. (0.534s)


In [29]:
#!python detect.py --weights yolov7.pt --conf 0.25  --source inference/images/cccd_resized_standard.jpg

Namespace(weights=['yolov7.pt'], source='inference/images/cccd_resized_standard.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.4.0+cu121 CUDA:0 (NVIDIA Graphics Device, 12008.875MB)

/home/phucnguyen/code/github_clone/YOLO/yolov7/models/experimental.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling.

In [13]:
if human != True:
    print("Else")

IF WE HAVE THE BOUNDING BOX, BASE ON THE FIX TEMPLATE OF CCCD, WE CAN APPROXIMATE THE AREA WHICH HAVE BASED INFORMATION OF CCCD

THEN WE WILL CUT THIS AREA OUT

In [25]:
fix_image_size = [1080, 681]

face_bounding_box = face_bounding_box

face_height = face_bounding_box[1][1] - face_bounding_box[0][1]
face_width = face_bounding_box[1][0] - face_bounding_box[0][0]

height_position_percent = 0.73

width_percent_differ = 2.25
height_percent_differ = 0.95

In [26]:
if face_height > 420 or face_width > 390:
    print("Human")

In [27]:
#def calculate_wanted_area(fix_image_size, face_bounding_box, height_position_percent, width_percent_differ, height_percent_differ, face_height, face_width):
text_bounding_box = [[0,0], [0,0]]

text_bounding_box[1][0] = face_bounding_box[1][0] + face_width * width_percent_differ

text_bounding_box[1][1] = face_bounding_box[1][1] - face_height * height_position_percent

text_bounding_box[0][0] = face_bounding_box[1][0]
text_bounding_box[0][1] = text_bounding_box[1][1] - face_height * height_percent_differ

In [28]:

# Extract the coordinates
top_left = text_bounding_box[0]
bottom_right = text_bounding_box[1]

# Calculate the other two corners
top_right = [bottom_right[0], top_left[1]]  # [c, b]
bottom_left = [top_left[0], bottom_right[1]]  # [a, d]

# Combine all four corners
all_corners = [top_left, top_right, bottom_right, bottom_left]

# Print the result
print("All corners:", all_corners)


All corners: [[312, 47.160000000000025], [847.5, 47.160000000000025], [847.5, 344.51], [312, 344.51]]


In [29]:
image = cv2.imread(image_path)

if image is None:
    print("Error: Could not load image.")
    exit()

# Function to crop regions based on bounding box
output_dir = "crop/output_regions"
os.makedirs(output_dir, exist_ok=True)


    # Convert points to numpy array
points_array = np.array(all_corners, dtype=np.int32)

# Get bounding rectangle (x_min, y_min, width, height)
x, y, w, h = cv2.boundingRect(points_array)

# Crop the region
cropped = image[y:y+h, x:x+w]

# Save the cropped region
output_path = f"{output_dir}/cut.jpg"
cv2.imwrite(output_path, cropped)
#print(f"Saved region '{name}' to {output_path}")


img = Image.open(output_path)
plt.imshow(img)
plt.axis('off')  # Optionally hide axes
plt.show()  # This will display the image

OCR WITH EASYOCR

In [31]:
result = reader.readtext('crop/output_regions/cut.jpg', detail = 0)
result

['CONG HOA XA HOI CHU NGHIA VIET NAM',
 'Độc lêP',
 'Tự do',
 'Hanh phúc',
 'SOCIALIST REPUBLIC OF VIET NAM',
 'Independence',
 'Freedom',
 'Happiness',
 'CĂN cƯỚC CÔNG DÂN',
 'Citizen Identity Card',
 'Só / No :',
 '048202007174']

DEFINE BASE INFORMATION ON CCCD

IF MATCH SOME OF THESE, THIS WILL VERY LIKELY A CCCD

In [32]:
pre_define_CCCD = ['conghoaxahoichunghiavietnam',
 'doclap',
 'tudo',
 'hanhphuc',
 'socialistrepublicofvietnam',
 'independence',
 'freedom',
 'happiness',
 'cancuocongdan',
 'citizenidentitycard',
 'so/no']

total_check = 11

In [33]:
import re
from unidecode import unidecode

# Function to normalize text
def normalize_vietnamese(text):
    # Remove accents using unidecode
    text = unidecode(text)
    # Convert to lowercase
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Remove extra spaces
    text = " ".join(text.split())
    text = text.replace(" ", "")
    
    return text

# Process the list
normalized_output = [normalize_vietnamese(item) for item in result]

# Print the result
print(normalized_output)

['conghoaxahoichunghiavietnam', 'doclep', 'tudo', 'hanhphuc', 'socialistrepublicofvietnam', 'independence', 'freedom', 'happiness', 'cancuoccongdan', 'citizenidentitycard', 'sono', '']


In [34]:
count = 0
for word in normalized_output:
    if word in pre_define_CCCD:
        count +=1
        
if count/11 > 0.5:
    print("Day la CCCD")
else:
    print('Day la Else')

Day la CCCD
